In [ ]:
"""Written for exporting data.

!! not updated according to latest API changes.

TODO:
    update it by looking at vis_or_export.ipynb
    Turn it into a cli_script for exporting csv and visuals.


"""

In [2]:
import numpy as np 
from numpy import linspace
import pandas as pd


from datetime import datetime,timedelta
from pathlib import Path



In [3]:
from nna.pre_process_func import read_queue
from nna.fileUtils import read_file_properties
from nna.labeling_utils import load_labels

from nna.visUtils import file2TableDict,createTimeIndex#,get_cycle,
from nna.visUtils import rawFile2Csv

from nna.fileUtils import list_files,standard_path_style


In [4]:
flder="/home/enis/projects/nna/data/"

# files_list_path=flder+"mp3files.txt"
# fileswlen_path = flder+"mp3files_wlen.txt"

# files_list_path=flder+"flacfiles.txt"
# fileswlen_path = flder+"flacfiles_wlen.txt"

# coordinates_fpath= flder + "NNA_recording_devices.csv"

In [7]:
file_properties_df=pd.read_pickle("../../data/prudhoeAndAnwr4photoExp_dataV1.pkl")

all_areas = sorted(pd.unique(file_properties_df.site_id.values))

In [8]:
file_properties_df

,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd
/tank/data/nna/real/prudhoe/17/2019/S4A10307_20190731_221602.flac,17,17,,S4A10307,221602,2019,07,31,prudhoe,2019-07-31 22:16:02,4438,2019-07-31 23:30:00
/tank/data/nna/real/prudhoe/12/2019/S4A10274_20190811_103000.flac,12,12,,S4A10274,103000,2019,08,11,prudhoe,2019-08-11 10:30:00,4560,2019-08-11 11:46:00
/tank/data/nna/real/prudhoe/27/2019/S4A10339_20190706_093000.flac,27,27,,S4A10339,093000,2019,07,06,prudhoe,2019-07-06 09:30:00,4560,2019-07-06 10:46:00
/tank/data/nna/real/prudhoe/30/2019/S4A10264_20190819_224602.flac,30,30,,S4A10264,224602,2019,08,19,prudhoe,2019-08-19 22:46:02,4438,2019-08-20 00:00:00
/tank/data/nna/real/prudhoe/27/2019/S4A10339_20190802_100000.flac,27,27,,S4A10339,100000,2019,08,02,prudhoe,2019-08-02 10:00:00,4560,2019-08-02 11:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...
/tank/data/nna/real/anwr/34/2019/S4A10327_20190703_223000.flac,34,34,,S4A10327,223000,2019,07,03,anwr,2019-07-03 22:30:00,4560,2019-07-03 23:46:00
/tank/data/nna/real/anwr/34/2019/S4A10327_20190703_234602.flac,34,34,,S4A10327,234602,2019,07,03,anwr,2019-07-03 23:46:02,4438,2019-07-04 01:00:00
/tank/data/nna/real/anwr/41/2019/S4A10273_20190705_000000.flac,41,41,,S4A10273,000000,2019,07,05,anwr,2019-07-05 00:00:00,4560,2019-07-05 01:16:00
/tank/data/nna/real/anwr/34/2019/S4A10327_20190705_000000.flac,34,34,,S4A10327,000000,2019,07,05,anwr,2019-07-05 00:00:00,4560,2019-07-05 01:16:00


In [12]:
# PARAMS
# FREQS to reduce results 
dataFreq="10S"
# freq="T"
freq="2H"

# clipping threshold
threshold=0.01  # if x <= then 0

# possible places to pick
# sorted(pd.unique(file_properties_df.site_id.values))
# areas to be visualized

# delete older than 2016
fromtime=datetime(2016, 1, 1, 0)

result_path="/scratch/enis/data/nna/real/"

selected_areas=all_areas[:]


# tag_set=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]
# tag_set=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

# model_tag_names=["CABLE","RUNNINGWATER","INSECT", "RAIN", "WATERBIRD", "WIND", "SONGBIRD", "AIRCRAFT"]
model_tag_names=["SONGBIRD",]

model_tag_names=["_"+i for i in model_tag_names]


In [16]:
model_tag_names

['_SONGBIRD']

In [13]:
#important to keep them in order
file_properties_df.sort_values(by=['timestamp'],inplace=True)

# delete olde r than 2016
fromtime=datetime(2016, 1, 1, 0)
file_properties_df=file_properties_df[file_properties_df.timestamp>=fromtime]


/scratch/enis/conda/envs/speechEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
len(file_properties_df)

30456

In [17]:
%%time

################# Saving 10S directly 
freq="10S"

globalcolumns=model_tag_names#selected_areas+weather_cols

globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)


saveCsvPath=Path("/home/enis/projects/nna/results/ExternalProject/Todd-Megan/Freq-10S/")
saveCsvPath = Path("/home/enis/projects/nna/results/test26102021/")

saveCsvPath.mkdir(parents=True,exist_ok=True)

csvFilesWritten,no_result_paths=rawFile2Csv(saveCsvPath,selected_areas,model_tag_names,
                                            globalindex,globalcolumns,file_properties_df,
                                            freq,dataFreq,result_path=result_path,prob2binaryFlag=False)


CPU times: user 2min 13s, sys: 11.8 s, total: 2min 25s
Wall time: 20min 46s


In [10]:
%%time

################# Saving as grouped by frequency  
freq="2H"


globalcolumns=model_tag_names#selected_areas+weather_cols

globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)

saveCsvPath=Path("/home/enis/projects/nna/results/ExternalProject/Todd-Megan/Freq-2H/")
saveCsvPath=Path("/home/enis/projects/nna/results/test26102020/Freq-2H/")

saveCsvPath.mkdir(parents=True,exist_ok=True)

df_dict,no_result_paths=file2TableDict(selected_areas,model_tag_names,globalindex,
                                       globalcolumns,file_properties_df,freq,dataFreq,
                                       result_path=result_path,prob2binaryFlag=False)


for key,val in df_dict.items():
    print(key)
    df_count,df_sums=val
    res=(df_sums/df_count).fillna(0.0)
    for col in res.columns:
        csvFilename="_".join([key,col[1:]+".csv"])
        res.to_csv(saveCsvPath/csvFilename,columns=[col],float_format='%.2f',index_label="TimeStamp",header=[col[1:]])


16-FishCreek2, do not have any files
20-Umiat, do not have any files
01-Itkillik
02-Colville2
03-OceanPt
04-Colville4
05-Colville5
06-Umiruk
07-IceRd
08-CD3
09-USGS
10-Nigliq1
11-Nigliq2
12-Anaktuvuk
13-Shorty
14-Rocky
15-FishCreek1
16-FishCreek2
17-FishCreek3
18-FishCreek4
19-Itkillik2
20-Umiat
CPU times: user 27.7 s, sys: 2.31 s, total: 30 s
Wall time: 2min 7s


1
